# PAC Learning Framework

In this section, I will concisely cover the Probably Approximately Correct (PAC) learning framework. 

## Motivation
In a sentence, the PAC framework quantifies the requirements (number of training points, time complexity, and space complexity) to approximately learn patterns in the data. THIS IS HUGE! For instance, a simple example is learning linear versus nonlinear maps between the input and outputs - linear patterns intuitively require less points to learn the pattern. Concretely, we could look at predicting height with weight data. If we desired a linear pattern, then we only need to search over $\mathbb{R}_+$ to find a parameter $\alpha$ such that $\text{height}=\alpha\cdot \text{weight}$. If instead we desire a nonlinear pattern, then think about the expansion of the space! We could fit polynomials, trigonometric, and the list goes on. I believe this should provide sufficient motivation for why we care deeply about quantifying the number of samples we need depending on the relationships we deem important (linear, nonlinear, indicator functions, exponentials, etc.). Now, I turn to mathematics for precision and clarity.

## Some Notations

- Let $\mathcal{X}$ denote the input space (includes all data excluding the output variable)

- Let $\mathcal{Y}$ denote the set of labels/output/target variables (as in [1], I limit discussion to binary outputs right now)

- Examples $(x,y)\in(\mathcal{X},\mathcal{Y})$ are assumed to be drawn independently and identically distributed (i.i.d.) according to some unknown distribution $\mathcal{D}$ 

- 

## Definitions

**Concept**: A concept $c:\mathcal{X}\rightarrow\mathcal{Y}$ is a mapping from $\mathcal{X}$ to $\mathcal{Y}$.

**Concept Class**: A concept class $\mathcal{C}$ is a set of concepts we may wish to learn. THESE 

## Examples

- Let us consider the features to be height, weight, age, ethnicity, and cancer (boolean) with the target being death (boolean). Then let us discuss concepts: for instance, all instances of Germans over 6' with weight >300lbs with cancer and over the age 55 get mapped to 1 i.e. they're dead (this is made up). This concept is thus an indicator function. We could consider a concept class to be a union of similar indicator variables. This type of concept class forms the basis for rule-based algorithms which will be discussed in a separate notebook.

- In the book, they discuss out of the ordinary examples - such as the concept class of triangles or circles or other geometric figures. While yes these are technically concept classes, they are sparsely used in practice.

## Definitions

**Generalization Error**: Given a hypothesis $h\in\mathcal{H}$, a target concept $c\in\mathcal{C}$, and an underlying distribution $D$, the generalization error or risk of $h$ is defined by 

$$R(h) = P_{x\sim D}\left[h(x)\neq c(x)\right] = \mathbb{E}_{x\sim D}\left[\mathbb{I}_{h(x)\neq c(x)}\right].$$

In words, it is the probability that the hypothesis does not align with the target concept. This is of course unknown as we do not know $D$; however, we can measure the empirical error of the hypothesis based on a labeled sample of the data.

**Empirical Error**: Given a hypothesis $h\in\mathcal{H}$, a target concept $c\in\mathcal{C}$, and a sample $S=(x_1,\dots,x_m)$, the empirical error or risk of $h$ is defined by 

$$\hat{R}(h) = \frac{1}{m}\sum\limits_{i=1}^m \mathbb{I}_{h(x_i)\neq c(x_i)}.$$

It is an emsemble of the error derived from the sample $S$.



# Sources

[1] "Foundations of Machine Learning" by Mohri et al., 2012